In [77]:
#One time run to import the project to your drive
!git clone https://github.com/KodeWil/final_project_AI_csv_Data.git

fatal: destination path 'final_project_AI_csv_Data' already exists and is not an empty directory.


In [0]:
def addData(files: str) -> list:
    allCsv = []
    for file in files:
      temp = open(file)
      tempDf = pd.read_csv(file)
      allCsv.append(tempDf)
    return allCsv

In [79]:
import pandas as pd 
import glob 
frames = []

#Create a list ('frames') with four nested lists, one per sensor. 
#Each nested list have all the samples (DF) for that sensor.
%cd /content/final_project_AI_csv_Data/phone/accel
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
%cd /content/final_project_AI_csv_Data/phone/gyro
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
%cd /content/final_project_AI_csv_Data/watch/accel
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
%cd /content/final_project_AI_csv_Data/watch/gyro
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
# 0 in frames = /phone/accel
# 1 in frames = /phone/gyro
# 2 in frames = /watch/accel
# 3 in frames = /watch/gyro

/content/final_project_AI_csv_Data/phone/accel
/content/final_project_AI_csv_Data/phone/gyro
/content/final_project_AI_csv_Data/watch/accel
/content/final_project_AI_csv_Data/watch/gyro


In [0]:
import sys
from IPython.display import Image, display
import copy
import random
import pandas as pd
import numpy as np

In [0]:
def printAll(value):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(value)

In [0]:
def activityType(activity : str) -> int:
    noHand = ['A', 'B', 'C', 'M', 'E']
    eating = ['H', 'I', 'J', 'K', 'L']
    if (activity in noHand):
      return 0
    elif (activity in eating):
      return 1
    else:
      return 2


In [0]:
def mapActivitiesTemp(activities):
  mapped = activities
  map(activityType, mapped)

In [0]:
def mapActivities(activities):
  noHand = ['A', 'B', 'C', 'M', 'E']
  eating = ['H', 'I', 'J', 'K', 'L']
  mapped = activities
  for i in range(0,len(activities)):
    if (activities[i] in noHand):
      mapped[i] = 0
    elif (activities[i] in eating):
      mapped[i] = 1
    else:
      mapped[i] = 2
  return mapped

In [0]:
#Directly related features are wrapped in intervals on a list
def selectFeatures(features) -> list:
  labels = copy.deepcopy(features)
  labels = labels.tolist()
  intervals = []
  atts = []
  intervals.append([2, 31]) 
  intervals.append([32, 34]) # {X,Y,Z}AVG 
  intervals.append([35, 37]) # {X,Y,Z}PEAK 
  intervals.append([38, 40]) # {X,Y,Z}ABSOLDEV 
  intervals.append([41, 43]) # {X,Y,Z}STANDDEV 
  intervals.append([44, 46]) # {X,Y,Z}VAR* 
  intervals.append([47, 85]) # FCC
  intervals.append([86, 88]) # COS
  intervals.append([89, 91]) # COR
  intervals.append([92, 92]) # resultant
  selected = []
  nFeatures = random.randint(0, 1)
  for i in range (0, nFeatures + 1):
    selected.append(i)

  for element in selected:
    ini, fin = intervals[element]
    if (ini != 92):
      for i in range(ini, fin + 1):
        atts.append(features[i])
    else:
      atts.append(features[92])
  return atts


In [0]:
#Kmeans clustering
def scoreK(values, ground, clusters):
  from sklearn.cluster import KMeans
  from sklearn import metrics
  # en n_clusters ponemos el número de actividades = 18
  kmeans = KMeans(n_clusters=clusters).fit(values)
  predicho = kmeans.predict(values)

  # adjusted_rand_score entre -1 y 1, mayor que cero es bueno, 1 es perfecto
  rand = metrics.adjusted_rand_score(ground, predicho)
  rand = round(rand, 4)
  
  # adjusted_mutual_info_score 0 y 1, 1 es perfecto
  mutual = metrics.adjusted_mutual_info_score(ground, predicho)
  mutual = round(mutual, 4)
  
  score = (rand, mutual)
  return score

In [0]:
#Agglomerative clustering
def scoreAgglo(values, ground, clusters):
  from sklearn.cluster import AgglomerativeClustering
  from sklearn import metrics
  
  # en n_clusters ponemos el número de actividades = 18
  aglo = AgglomerativeClustering(n_clusters= clusters).fit(values)
  predicho = aglo.labels_

  # adjusted_rand_score entre -1 y 1, mayor que cero es bueno, 1 es perfecto
  rand = metrics.adjusted_rand_score(ground, predicho)
  rand = round(rand, 4)

  # adjusted_mutual_info_score 0 y 1, 1 es perfecto
  mutual = metrics.adjusted_mutual_info_score(ground, predicho)
  mutual = round(mutual, 4)

  score = (rand, mutual)
  return score

In [0]:
#'' clustering
def scoreSpecs(values, ground, clusters):

  from sklearn.cluster import AgglomerativeClustering
  from sklearn import metrics

  # en n_clusters ponemos el número de actividades = 18
  aglo = AgglomerativeClustering(n_clusters= clusters).fit(values)
  predicho = aglo.labels_

  # adjusted_rand_score entre -1 y 1, mayor que cero es bueno, 1 es perfecto
  rand = metrics.adjusted_rand_score(ground, predicho)
  
  # adjusted_mutual_info_score 0 y 1, 1 es perfecto
  mutual = metrics.adjusted_mutual_info_score(ground, predicho)
  mutual = round(mutual, 4)
  score.append(rand)
  score.append(mutual)
  return score

In [0]:
#Calculate adjusted_rand_score and mutual_info_score for each sample and each clustering method
def calcScore(sample):
  
  #Set values to evalute clustering score
  ground_truth = sample["ACTIVITY"].values
  lookupTable, ground_truth = np.unique(ground_truth, return_inverse=True)
  features = selectFeatures(sample.columns)
  X = sample[features].values

  #Evaluate clustering method

  #Kmeans
  temRand,temMutual = scoreK(X, ground_truth, 18)
  scoresK = (temRand, temMutual)

  #Agglomerative
  temRand,temMutual = scoreAgglo(X, ground_truth, 18)
  scoresAgglo = (temRand, temMutual)
  scores = [scoresK, scoresAgglo]
  return scores

In [0]:
#rank is a list that contains 50 list (one per sample), a those 50 lists have a list for clustering method.
# The most nested list contain the values for the two calculated scores
def sensorScores(samplesDf):
  rank = list(map(calcScore, samplesDf))
  scoreFrame = pd.DataFrame(rank)
  cluster = ['Kmeans', 'Agglomerative']
  scoreFrame.columns = cluster
  return scoreFrame

In [0]:
allRanks = list(map(sensorScores, frames))


In [95]:
print(allRanks[0])

              Kmeans     Agglomerative
0   (0.3469, 0.5548)  (0.3391, 0.5617)
1   (0.3151, 0.5505)   (0.331, 0.5646)
2   (0.2356, 0.4955)  (0.2416, 0.4976)
3   (0.5352, 0.7524)    (0.542, 0.765)
4   (0.2547, 0.5134)  (0.2416, 0.4976)
5   (0.2277, 0.4901)  (0.2416, 0.4976)
6   (0.2631, 0.5041)  (0.2416, 0.4976)
7   (0.2412, 0.4967)  (0.2416, 0.4976)
8   (0.2295, 0.4906)  (0.2416, 0.4976)
9   (0.5921, 0.7591)   (0.621, 0.7638)
10  (0.2076, 0.4144)  (0.2311, 0.4607)
11   (0.375, 0.5882)  (0.3335, 0.5715)
12  (0.5692, 0.7511)  (0.5721, 0.7581)
13   (0.4131, 0.612)   (0.425, 0.6313)
14  (0.4286, 0.6387)   (0.393, 0.6245)
15  (0.6021, 0.7465)  (0.5967, 0.7503)
16  (0.1437, 0.3535)  (0.1402, 0.3485)
17  (0.6342, 0.7806)  (0.6798, 0.8155)
18   (0.259, 0.5229)  (0.3025, 0.5605)
19  (0.3028, 0.5281)  (0.3134, 0.5339)
20  (0.2953, 0.5376)   (0.317, 0.5688)
21  (0.4399, 0.6259)  (0.4182, 0.6235)
22  (0.3525, 0.5617)  (0.3626, 0.5643)
23  (0.7361, 0.8441)  (0.7364, 0.8443)
24  (0.5144, 0.7311)    (

In [0]:
# Wrap activities in three categories as suggested in dataset description 
# seleccionar = ["ACTIVITY"]
# X = allData[seleccionar].values
# activityType = copy.deepcopy(X)
# activityType = mapActivities(activityType)
# allData.insert(0, "actType", activityType, True)